In [1]:
from scipy.io import loadmat
import numpy as np
import math
import matplotlib.pyplot as plt
import scipy.optimize as opt

In [2]:
def backprop (params_rn, num_entradas, num_ocultas, num_etiquetas, X, Y, reg ):

    #Obtenemos theta1 y theta2
    theta1 = np.reshape(params_rn [:num_ocultas * (num_entradas + 1)],
    (num_ocultas, (num_entradas + 1) ) )
    theta2 = np.reshape(params_rn [num_ocultas * (num_entradas + 1):],
    ( num_etiquetas, (num_ocultas + 1) ) )
    
    #Añadimos una columna de unos a las X
    unos = np.ones([X.shape[0],1])
    _X = np.concatenate((unos,X),axis=1)    
    
    #Hallamos la hipótesis para las theta dadas, esto es necesario para el calculo del coste
    h = hipotesis(_X,theta1,theta2).T

    #Calculamos el coste con regularización
    coste = coste_Reg(h, Y, theta1, theta2, num_etiquetas, reg)
    grad = retro_propagacion(_X, Y, theta1, theta2, reg, num_etiquetas)

    return coste, grad

In [3]:
def coste_NR(h,Y,num_etiquetas):
    
    coste=0
    m = Y.shape[0]
    
    for i in range(num_etiquetas):
       
        _Y = (Y==i+1)*1
        coste += sum(-_Y*(np.log(h[i])) - (1-_Y)*(np.log(1-h[i])))
        
    return coste/m

In [4]:
def coste_Reg(h, Y, theta1, theta2, num_etiquetas, reg):
    
    m = Y.shape[0]
    theta1[0] = 0.0
    theta2[0] = 0.0
    
    coste = coste_NR(h,Y,num_etiquetas)
    coste += (sum(sum(theta1**2)) + sum(sum(theta2**2))) * (reg/m*2)
    
        
    return coste

In [5]:
def sigmoide(Z):
  
    return 1/(1 + (math.e ** -Z))

 
def derivada_sig(Z):
    
    return sigmoide(Z)*(1-sigmoide(Z))
        
        
def hipotesis(X,theta1,theta2):
    
    #Capa primera - Capa oculta
    a_1 = sigmoide(X.dot(theta1.T))
    unos = np.ones([a_1.shape[0],1])
    
    #Capa oculta - Capa salida
    a_1 = np.concatenate((unos,a_1),axis=1)
    a_2 = sigmoide(a_1.dot(theta2.T))
    
    return a_2

In [6]:
def pesosAleatorios(L_in, L_out):
    
    e_ini = 0.12
    pesos = np.random.uniform(-e_ini,e_ini,size=(L_out, L_in+1))
    return pesos

In [7]:
def retro_propagacion(X, Y, theta1, theta2, reg, num_etiquetas):
        
    y = np.zeros((Y.shape[0],num_etiquetas))
    m = Y.shape[0]
    
    for i in range (num_etiquetas):

        y[i, Y[i]-1] = 1 
        
     #Capa primera - Capa oculta
    a_1 = sigmoide(X.dot(theta1.T))
    unos = np.ones([a_1.shape[0],1])
    
    #Capa oculta - Capa salida
    a_2 = np.concatenate((unos,a_1),axis=1)
    a_3 = sigmoide(a_2.dot(theta2.T))
    
    
    sigma3 = a_3 - y
    sigma2 = sigma3.dot(theta2)[:,1:]
    sigma2 = sigma2 * derivada_sig(X.dot(theta1.T))
    delta1 = sigma2.T.dot(X)
    delta2 = sigma3.T.dot(a_2)

    grad1 = delta1/m
    grad2 = delta2/m
    
    theta1[:,1]=0
    theta2[:,1]=0
    grad1 = grad1+(reg/m)*theta1
    grad2 = grad2+(reg/m)*theta2
    
    #Devolvemos el gradiente como una sola linea    
    return np.concatenate((np.ravel(grad1),np.ravel(grad2))) 

In [8]:
def aprendizajeRN(X, Y, params, num_entradas, num_ocultas, num_etiquetas, lamb):
    
    
    m = Y.shape[0]
    
    #Función que entrena la red neuronal
    fmin = opt.minimize(fun=backprop, x0=params, 
                        args=(num_entradas, num_ocultas, num_etiquetas, X, Y, lamb),
                        method='TNC', jac=True, options={'maxiter':30})
    
    #Obtenemos las theta del entrenamiento
    theta1 = np.reshape(fmin.x [:num_ocultas * (num_entradas + 1)],
    (num_ocultas, (num_entradas + 1) ) )
    theta2 = np.reshape(fmin.x [num_ocultas * (num_entradas + 1):],
    ( num_etiquetas, (num_ocultas + 1) ) )
    
    unos = np.ones([X.shape[0],1])
    _X = np.concatenate((unos,X),axis=1)
    
    #Comprobamos la fidelidad del entrenamiento con los casos de prueba
    res = hipotesis(_X,theta1,theta2)
    
    result = np.argmax(res,axis=1) 
    
    result = (result+1 == (Y))*1
    
    porcentaje = (sum(result)*100)/m

    print("Término de regularización: ", lamb)
    print("Númeo de iteraciones: ", 30)
    print("Porcentaje de acierto: ", porcentaje)

In [9]:
def main1():
    
    #Almacenamos los datos leídos en X, Y.
    data = loadmat('ex4data1.mat')
    Y = data['y']
    X = data['X']
    Y = np.array(Y.T)[0]
    
    
    weights = loadmat('ex4weights.mat')
    theta1, theta2 = weights ['Theta1'] , weights ['Theta2']

    # Theta1 e s de dimensión 25 x 401
    # Theta2 e s de dimensión 10 x 26
    
    
    theta_vec = np.concatenate((np.ravel(theta1),np.ravel(theta2)))
    num_entradas = theta1.shape[1]-1
    num_ocultas = theta1.shape[0]
    
    aprendizajeRN(X,Y,theta_vec,num_entradas, num_ocultas, 10, 100)

    
   


main1()

Término de regularización:  100
Númeo de iteraciones:  30
Porcentaje de acierto:  94.1
